In [ ]:
# === Environment Setup ===
import os, sys, math, time, random, json, textwrap, warnings
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy.optimize import fsolve
from scipy.integrate import solve_ivp
from numba import njit, prange
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import display, Markdown

# --- Configuration ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({'font.size': 12, 'figure.figsize': (10, 6), 'figure.dpi': 130,
                     'axes.titlesize': 'x-large', 'axes.labelsize': 'large',
                     'xtick.labelsize': 'medium', 'ytick.labelsize': 'medium'})
np.set_printoptions(suppress=True, linewidth=120, precision=4)

# --- Utility Functions ---
def note(msg, **kwargs):
    display(Markdown(f"<div class='alert alert-info'>📝 {textwrap.fill(msg, width=100)}</div>"))
def sec(title):
    print(f"\n{100*'='}\n| {title.upper()} |\n{100*'='}")

note("Environment initialized.")

# Part 4: Macroeconomic Models
## Chapter 4.02: Neoclassical and Endogenous Growth Models

### Table of Contents
1.  [The Solow-Swan Model of Exogenous Growth](#1.-The-Solow-Swan-Model-of-Exogenous-Growth)
    *   [1.1 The Fundamental Law of Motion](#1.1-The-Fundamental-Law-of-Motion)
    *   [1.2 The Golden Rule and Optimal Savings](#1.2-The-Golden-Rule-and-Optimal-Savings)
2.  [The Ramsey-Cass-Koopmans (RCK) Model of Optimal Growth](#2.-The-Ramsey-Cass-Koopmans-(RCK)-Model-of-Optimal-Growth)
    *   [2.1 The Hamiltonian and Necessary Conditions](#2.1-The-Hamiltonian-and-Necessary-Conditions)
    *   [2.2 Phase Diagram Analysis](#2.2-Phase-Diagram-Analysis)
3.  [The Stochastic RCK Model](#3.-The-Stochastic-RCK-Model)
    *   [3.1 The Model with Productivity Shocks](#3.1-The-Model-with-Productivity-Shocks)
    *   [3.2 Solving with Value Function Iteration](#3.2-Solving-with-Value-Function-Iteration)
4.  [Endogenous Growth: The AK Model](#4.-Endogenous-Growth:-The-AK-Model)
5.  [Chapter Summary](#5.-Chapter-Summary)
6.  [Exercises](#6.-Exercises)

### Introduction: The Engines of Economic Growth
The study of economic growth seeks to answer some of the most fundamental questions in economics: Why are some countries so rich while others are so poor? What are the engines of sustained increases in living standards? This chapter explores the foundational frameworks for analyzing these questions.

We begin with **neoclassical growth models**, which are built on the assumption of diminishing returns to capital. These models provide the crucial insight that long-run growth in living standards must be driven by exogenous technological progress.
- **The Solow-Swan Model**: The canonical starting point, which treats the saving rate as exogenous.
- **The Ramsey-Cass-Koopmans (RCK) Model**: A richer model that endogenizes the saving decision via intertemporal utility maximization.
- **The Stochastic RCK Model**: An extension that introduces aggregate productivity shocks, requiring a stochastic dynamic programming solution.

Finally, we introduce **endogenous growth theory**, which seeks to explain technological progress itself as the outcome of economic decisions, using the simple **AK model** as the canonical example.

### 1. The Solow-Swan Model of Exogenous Growth

The Solow-Swan model, developed independently by Robert Solow and Trevor Swan in 1956, is the starting point for nearly all modern analyses of economic growth. Solow's work on the model earned him the 1987 Nobel Prize in Economics. Its enduring power lies in its simplicity and the starkness of its central conclusion: that capital accumulation alone, due to diminishing returns, cannot sustain long-run growth in living standards. Instead, sustained per-capita growth must be driven by exogenous technological progress.

#### 1.1 The Fundamental Law of Motion
The model analyzes the economy in terms of quantities per unit of effective labor, $k = K/(AL)$. The central dynamic equation describes the evolution of $k$ as the difference between actual investment and break-even investment:
$$ \dot{k} = s f(k) - (n + g + \delta)k $$
where $s$ is the saving rate, $f(k)$ is the production function (e.g., $k^\alpha$), $n$ is population growth, $g$ is technological progress, and $\delta$ is depreciation. The economy reaches a **steady state** ($k^*$) when $\dot{k} = 0$, which occurs where $s f(k^*) = (n + g + \delta)k^*$. Due to diminishing returns to capital, this steady state is globally stable.

#### 1.2 The Golden Rule and Optimal Savings
The **Golden Rule** level of capital, $k^*_{gold}$, is the level that maximizes steady-state consumption. It occurs where the marginal product of capital equals the effective depreciation rate: $ f'(k^*_{gold}) = n + g + \delta $. For a Cobb-Douglas production function, this corresponds to a saving rate equal to the capital share, $s_{gold} = \alpha$.

In [ ]:
sec("The Solow-Swan Model and the Golden Rule")

# --- Model Parameters ---
# s: The exogenous saving rate.
# n: The growth rate of the labor force.
# g: The rate of labor-augmenting technological progress.
# delta: The depreciation rate of capital.
# alpha: The capital share in the Cobb-Douglas production function f(k) = k^alpha.
def analyze_solow(s=0.3, n=0.01, g=0.015, delta=0.05, alpha=0.33):
    k_star = (s / (n + g + delta))**(1 / (1 - alpha))
    k_golden = (alpha / (n + g + delta))**(1 / (1 - alpha))
    
    k_grid = np.linspace(0, max(k_star, k_golden) * 1.8, 200)
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(k_grid, s * k_grid**alpha, lw=2.5, label='$s f(k)$ (Actual Investment)')
    ax.plot(k_grid, (n + g + delta) * k_grid, lw=2.5, label='$(n+g+\delta)k$ (Break-even Inv.)')
    ax.plot(k_star, s * k_star**alpha, 'ro', ms=10, label=f'Steady State $k^*={k_star:.2f}$')
    ax.axvline(k_golden, color='gold', ls='--', lw=3, label=f'Golden Rule $k_{{gold}}^*={k_golden:.2f}$')
    ax.set_title('Solow Model: Steady State Determination')
    ax.set(xlabel='Capital per Effective Worker, $k$', ylabel='Investment'); ax.legend()
    plt.show()

analyze_solow()

### 2. The Ramsey-Cass-Koopmans (RCK) Model of Optimal Growth

While the Solow model provides deep insights, its assumption of a constant, exogenous saving rate is a significant shortcoming. The Ramsey-Cass-Koopmans (RCK) model, developed by Frank Ramsey in 1928 and refined by David Cass and Tjalling Koopmans in the 1960s, addresses this by endogenizing the saving decision. It does so by explicitly modeling the intertemporal consumption-saving choice of a representative household that seeks to maximize its lifetime utility. This provides a micro-foundation for the aggregate saving behavior of the economy.

A key prediction of the neoclassical models (both Solow and RCK) is **conditional convergence**: the idea that poorer countries should grow faster than richer countries, *conditional* on having the same steady-state characteristics (like saving rates, technology, and population growth). The models predict that the further an economy is from its steady state, the faster it will grow.

#### 2.1 The Hamiltonian and Necessary Conditions
The RCK model endogenizes the saving decision by deriving it from a representative household's intertemporal utility maximization problem. The planner chooses the path of consumption $c_t$ to maximize lifetime utility subject to the capital accumulation constraint. We solve this using the **current-value Hamiltonian**:
$$ \mathcal{H}(k, c, \lambda) = u(c) + \lambda [f(k) - (n+g+\delta)k - c] $$
The necessary first-order conditions for an optimum are:
1. $\frac{\partial \mathcal{H}}{\partial c} = 0 \implies u'(c) = \lambda$ (Optimality condition for consumption)
2. $\dot{\lambda} = (\rho-n)\lambda - \frac{\partial \mathcal{H}}{\partial k} \implies \dot{\lambda} = (\rho-n)\lambda - \lambda[f'(k) - (n+g+\delta)]$ (Co-state equation)
3. The original law of motion for capital, $\dot{k}$.
4. The **Transversality Condition**: $\lim_{t \to \infty} e^{-(\rho-n)t} \lambda_t k_t = 0$. This condition prevents the household from accumulating capital forever without consuming it.

Combining these yields the two-dimensional system in $(k, c)$ that defines the equilibrium dynamics.

#### 2.2 Phase Diagram Analysis
The equilibrium is described by a system of two differential equations in $k$ and $c$. The system has a unique non-trivial steady state $(k^*, c^*)$ which is a **saddle point**. This has a crucial economic interpretation: for any given initial capital stock $k_0$, there is a *single* initial consumption level $c_0$ that will guide the economy to its long-run steady state. This unique trajectory is called the **stable saddle path**.

- If initial consumption is **too high** (above the saddle path), the household consumes so much that it fails to invest enough to maintain its capital stock, leading to an unsustainable path where capital and consumption eventually fall to zero.
- If initial consumption is **too low** (below the saddle path), the household over-saves, accumulating capital indefinitely but never enjoying its fruits. This violates the transversality condition.

The phase diagram is a powerful tool for visualizing these dynamics.

In [ ]:
sec("RCK Model Phase Diagram and Transitional Dynamics")
class RCKModel:
    """
    Implements the Ramsey-Cass-Koopmans (RCK) model.
    Parameters:
      - alpha: Capital share
      - delta: Depreciation rate
      - rho: Subjective discount rate (impatience)
      - theta: Coefficient of relative risk aversion (inverse of intertemporal elasticity of substitution)
      - n: Population growth rate
      - g: Technological progress rate
    """
    def __init__(self, alpha=0.33, delta=0.05, rho=0.02, theta=2.0, n=0.01, g=0.015):
        self.alpha, self.delta, self.rho, self.theta, self.n, self.g = alpha, delta, rho, theta, n, g
        self.k_ss, self.c_ss = self._solve_steady_state()
    
    def _solve_steady_state(self):
        k_ss = (self.alpha / (self.delta + self.rho + self.theta * self.g))**(1 / (1 - self.alpha))
        c_ss = k_ss**self.alpha - (self.n + self.g + self.delta) * k_ss
        return k_ss, c_ss

    def system_dynamics(self, t, z): 
        k, c = z
        if k < 1e-6 or c < 1e-6: return [0,0]
        k_dot = k**self.alpha - (self.n + self.g + self.delta) * k - c
        c_dot = c / self.theta * (self.alpha * k**(self.alpha - 1) - self.delta - self.rho - self.theta * self.g)
        return [k_dot, c_dot]

rck = RCKModel()
k_range, c_range = np.meshgrid(np.linspace(0.1, rck.k_ss*2, 20), np.linspace(0.1, rck.c_ss*1.8, 20))
k_dot, c_dot = rck.system_dynamics(0, [k_range, c_range])

fig, ax = plt.subplots(figsize=(10, 8))
ax.streamplot(k_range, c_range, k_dot, c_dot, density=1.4, color='gray', linewidth=0.8)
k_null_locus = k_range[0,:]**rck.alpha - (rck.n + rck.g + rck.delta) * k_range[0,:]
ax.plot(k_range[0,:], k_null_locus, 'b-', lw=2.5, label='$\dot{k}=0$')
ax.axvline(rck.k_ss, color='orange', lw=2.5, label='$\dot{c}=0$')
ax.plot(rck.k_ss, rck.c_ss, 'ro', ms=12, label='Steady State')
ax.set_title('RCK Model Phase Diagram'); ax.set(xlabel='k', ylabel='c', xlim=(0, rck.k_ss*2), ylim=(0, rck.c_ss*1.8)); ax.legend()
plt.show()

### 3. The Stochastic RCK Model
We now introduce aggregate productivity shocks. The production function becomes $Y_t = Z_t K_t^\alpha (AL_t)^{1-\alpha}$, where the technology shock $Z_t$ follows a Markov process. This transforms the problem into a stochastic dynamic programming problem, which we can solve with VFI.

In [ ]:
sec("Solving the Stochastic RCK Model with VFI")
# --- Model Parameters ---
# K_GRID: The grid of possible capital stock levels.
# Z_GRID: The possible states for the aggregate productivity shock Z.
# P_TRANS: The Markov transition matrix for the productivity shock Z.
# BETA: The discount factor.
# ALPHA: The capital share.
# DELTA: The depreciation rate.
# THETA: The coefficient of relative risk aversion.
K_GRID = np.linspace(0.1, 40, 150)
Z_GRID, P_TRANS = np.array([0.95, 1.05]), np.array([[0.9, 0.1], [0.1, 0.9]])
BETA, ALPHA, DELTA, THETA = 0.96, 0.33, 0.05, 2.0

@njit
def u(c, theta=THETA): return (c**(1 - theta) - 1) / (1 - theta)

@njit(parallel=True)
def bellman_stochastic_rck(V, k_grid, z_grid, p_trans, beta, delta, alpha):
    V_new, policy = np.empty_like(V), np.empty_like(V)
    # The expected value function, EV(z, k) = E[V(z', k) | z], is calculated by 
    # multiplying the current value function by the transition matrix.
    EV = p_trans @ V
    
    for i in prange(len(z_grid)):
        z = z_grid[i]
        ev_interp = np.interp(k_grid, k_grid, EV[i, :])
        for j in range(len(k_grid)):
            k = k_grid[j]
            max_val, best_k_prime = -1e12, k_grid[0]
            k_prime_choices = k_grid[k_grid <= z*k**alpha + (1-delta)*k]
            for k_prime in k_prime_choices:
                c = z * k**alpha + (1 - delta) * k - k_prime
                val = u(c) + beta * np.interp(k_prime, k_grid, ev_interp)
                if val > max_val:
                    max_val, best_k_prime = val, k_prime
            V_new[i, j], policy[i, j] = max_val, best_k_prime
    return V_new, policy

V_init = np.zeros((len(Z_GRID), len(K_GRID)))
V_star, policy_star = V_init, V_init
for i in range(200):
    V_new, policy_new = bellman_stochastic_rck(V_star, K_GRID, Z_GRID, P_TRANS, BETA, DELTA, ALPHA)
    if np.max(np.abs(V_star - V_new)) < 1e-6: 
        print(f"Converged in {i} iterations."); break
    V_star, policy_star = V_new, policy_new

fig, ax = plt.subplots()
ax.plot(K_GRID, policy_star[0,:], label='Policy (Low Z)')
ax.plot(K_GRID, policy_star[1,:], label='Policy (High Z)')
ax.plot(K_GRID, K_GRID, 'k--', label='45-degree line')
ax.set_title('Stochastic RCK: Optimal Savings Policy'); ax.set_xlabel('k'); ax.set_ylabel("k'"); ax.legend()
plt.show()

### 4. Endogenous Growth: The AK Model
The neoclassical models predict that long-run growth in living standards is driven by exogenous technological progress. **Endogenous growth theory** seeks to explain this progress as the outcome of economic decisions. The simplest model is the **AK model**.

The key assumption is that the production function does not have diminishing returns to capital, $Y = AK$. Here, $K$ is interpreted broadly to include physical, human, and knowledge capital. The Euler equation simplifies to:
$$ \frac{\dot{c}}{c} = \frac{1}{\theta}(A - \delta - \rho) $$
If the parameters satisfy $A > \delta + \rho$, the economy exhibits a positive, constant growth rate determined *endogenously* by preferences and technology, without any need for an exogenous trend.

### 5. Chapter Summary
- **Exogenous vs. Endogenous Growth:** Neoclassical models (Solow, RCK) explain capital accumulation but rely on exogenous technological progress for long-run growth in living standards. Endogenous growth models (like the AK model) generate sustained growth by removing the assumption of diminishing returns to capital.
- **Solow Model:** Provides the fundamental insight that savings and capital accumulation cannot sustain long-run per-capita growth due to diminishing returns.
- **RCK Model:** Endogenizes the saving decision, providing a richer, microfounded model of growth dynamics. Its saddle-path stability is a common feature in dynamic general equilibrium models.
- **Stochastic Models:** Introducing aggregate shocks transforms the problem into a stochastic DP problem, which can be solved with the methods from the previous module.

### 6. Exercises

1.  **Golden Rule Savings Rate:** In the Solow model, show mathematically that for a Cobb-Douglas production function, the Golden Rule saving rate is equal to the capital share, `alpha`.

2.  **Comparative Dynamics (Impatience):** In the RCK model, households become more impatient (`rho` increases from 0.02 to 0.04). How does this affect the steady-state capital stock $k^*$ and consumption $c^*$? Explain the economic intuition.

3.  **Transversality Condition:** What is the economic intuition behind the transversality condition in the RCK model? What kind of suboptimal behavior does it rule out?

4.  **Taxation in the RCK Model:** Introduce a constant tax $\tau$ on capital income into the RCK model. The household's after-tax return on capital is now $(1-\tau)(f'(k) - \delta)$. How does this tax affect the $\dot{c}=0$ nullcline and the steady-state level of capital, $k^*$?

5.  **AK Model Dynamics:** For the AK model, what happens if $A < \delta + \rho$? What is the growth rate of consumption? Provide an economic intuition for this result.